### 1.1 Creación de un entorno virtual

In [ ]:
#conda create -n nombre_del_entorno python=3.9

### 1.2 Activacion del entorno

In [ ]:
#conda activate nombre_del_entorno python

## 2 Creacion de un DataSet Propio

Dado que se busca hacer un detector personalizado es muy probable que sea necesario tener nuestro propio set de datos para el entrenamiento del modelo, es posible que existan algunos datasets ya etiquetados que otras personas ya hayan etiquetado. Sin embargo para nustro caso particular deseabamos detectar algunos prodcutos de una tienda de conveniencia, al ser algo tan particular no encontramos otro recurso mas que la creación de nuestro dataset.

Esto perse no es algo negativo, de hecho construir el dataset con imagenes que luego procesara el modelo le da un mejor desempeño en esas condciones, sin embargo su desempeño podra variar al variar esas condiciones particulares. 

Cabe mencionar que la creación de un dataset propio implica un trabajo adicional, dado que implica otros procesos. A continuación presentamos algunas maneras de crear estos datasrts propios


### 2.1 Recolección de imagenes

#### 2.1.1 Dataset a partir de Internet
Una posible solución es simplemente descargar las imagenes de internet. Esto nos brindara una gran cantidad de imagenes con gran diversidad

Para hacer esto usaremos simple_image_download.


#pip install simple_image_download==0.4

Luego usaremos el siguiente script

In [ ]:
from simple_image_download import simple_image_download as simp

response =simp.simple_image_download

number_images=100                #Numero de imagenes a decargar palabra clave
keywords = ["hatsu azul 400 ml"] #Prompt o busqueda de google a hacer, puede poner varias palabras clave

for kw in keywords:
    response().download(kw,number_images)

Este script creara una carpeta llamada simple_images, dentro de la carpeta creara una carpeta por cada una de las keywords, una vez genere las carpetas traslade todas las imagenes a otra carpeta, por ejemplo una carpeta llamada data o images

In [ ]:
2

#### 2.1.2 Dataset a partir de captura de video - modo video
Otra posibilidad es crear el dataset a partir de una captura de video, es decir que cada cuadro o algunos de los cuadro del video sean imagenes para el entrenamiento del modelo

Para esto usaremos el siguiente script


In [ ]:
import cv2
import os
import time
import uuid

IMAGES_PATH = os.path.join('data','captured_images') #Declaramos la ruta donde guardaremos las imagenes capturadas
number_images = 30 #Declaramos la cantidad de iamgenes a capturar

cap = cv2.VideoCapture(0) # El numero indica la camara a usar, 1 suele ser una webcam externa mientras0 es la incorporada

time.sleep(1.5)  #tiempo entre imagenes capturadas

for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'): #Se indica que con q se cierra la ventana
        break
cap.release()
cv2.destroyAllWindows()

#### 2.1.3 Dataset a partir de captura de video - modo captura
Otra posibilidad es crear el dataset a partir de una captura de video, es decir que cada cuadro o algunos de los cuadro del video sean imagenes para el entrenamiento del modelo, a diferencia del modo anterior donde se capturan las imagnes seguido, en este modo se selecciona en que momento desea realizar la captura.

Para esto usaremos el siguiente script que captura la imagen al presionar la tecla s, se termina al salir(Preisonar tecla q ) o cuando se capture el numero de imagenes deseado.


In [ ]:
import cv2
import os
import time
import uuid

# Ruta donde se guardarán las fotos
images_path = os.path.join('data','captured_images')



# Función para capturar varias fotos y guardarlas en el directorio especificado
def capture_images():
    # Creamos la ventana donde se mostrará la imagen
    cv2.namedWindow("Captura de imagen", cv2.WINDOW_NORMAL)
    
    # Inicializamos la cámara
    cap = cv2.VideoCapture(1)
    
    # Contador para nombrar las imágenes capturadas
    count = 0
    number_images = 30 #Declaramos la cantidad de iamgenes a capturar
    
    while True:
        # Capturamos una imagen de la cámara
        ret, frame = cap.read()

        new_size = (640, 480)
        frame = cv2.resize(frame, new_size)
        
        # Mostramos la imagen capturada en la ventana
        cv2.imshow("Captura de imagen", frame)
        
        # Esperamos a que se presione una tecla
        key = cv2.waitKey(1)
        
        # Si se presiona la tecla "s", guardamos la imagen actual en el directorio especificado
        if key == ord("s"):
            count += 1
            filename = os.path.join(images_path,f'{str(uuid.uuid1())}.jpg')
            cv2.imwrite(filename, frame)
            print("Imagen guardada como", filename)
        
        # Si se presiona la tecla "q", salimos del bucle
        elif key == ord("q") or count>=number_images:
            break
    
    # Liberamos los recursos de la cámara y cerramos la ventana
    cap.release()
    cv2.destroyAllWindows()

# Llamamos a la función para capturar las fotos
capture_images()


### 2.2 Etiquetado de Imagenes
Una vez recolectadas las imagenes la siguiente etapa es etiquetar las imagenes, esto se refiere a indicar que objetos se encuentran en la imagen y cuales son sus pocisiones.

Para tal fin usaremos el paquete labelImg

In [ ]:
#pip install labelImg